In [1]:
import numpy as np
import pandas as pd
import cornac
from cornac.eval_methods import CrossValidation, RatioSplit
from cornac.data import Reader
from cornac.data import Dataset
from cornac.hyperopt import Discrete, GridSearch
from tabulate import tabulate

# Read and load data

In [2]:
# Init cornac reader object
reader = Reader()
# Read both the datasets from the files using cornac
movielens_data = reader.read(fpath="../data/u.data", sep="\t")
pda_data = reader.read(fpath="../data/train-PDA2018.csv", sep=",", skip_lines=1)
print("Movielens")
print(movielens_data[:5])
print()
print("PDA")
print(pda_data[:5])

Movielens
[('196', '242', 3.0), ('186', '302', 3.0), ('22', '377', 1.0), ('244', '51', 2.0), ('166', '346', 1.0)]

PDA
[('5', '648', 5.0), ('5', '1394', 5.0), ('5', '3534', 5.0), ('5', '104', 4.0), ('5', '2735', 5.0)]


In [3]:
# Build the datasets (This is similar to how surprise uses build_full_trainset)
movielens_dataset = Dataset.build(movielens_data)
pda_dataset = Dataset.build(pda_data)
# Print out some basic information about the datasets
print("General information on the training sets we will be using \n")
print("1) Number of items in each dataset", " ML100k:", movielens_dataset.num_items, "PDA:", pda_dataset.num_items)
print("2) Number of users in each dataset", " ML100k:", movielens_dataset.num_users, "PDA:", pda_dataset.num_users)
print("3) Number of ratings in each dataset", " ML100k:", movielens_dataset.num_ratings, "PDA:", pda_dataset.num_ratings)
print("4) Mean rating", " ML100k:", movielens_dataset.global_mean, "PDA:", movielens_dataset.global_mean)

General information on the training sets we will be using 

1) Number of items in each dataset  ML100k: 1682 PDA: 1824
2) Number of users in each dataset  ML100k: 943 PDA: 5690
3) Number of ratings in each dataset  ML100k: 100000 PDA: 470711
4) Mean rating  ML100k: 3.52986 PDA: 3.52986


# BPR Model

In [4]:
# Now we have data that is ready to be fed to the model

# First of let's define the BPR model using cornac. Cornac gives us the possibility to tweak a lot of hyperparameters
# We'll be using the default ones and then run a grid search to find out the best ones
bpr_model = cornac.models.BPR(
    k=10,
    learning_rate=0.001,
    lambda_reg=0.001
)

In [5]:
# Next we'll define the evaluation metrics for this model
rec_5 = cornac.metrics.Recall(k=5)
pre_5 = cornac.metrics.Precision(k=5)
rec_10 = cornac.metrics.Recall(k=10)
pre_10 = cornac.metrics.Precision(k=10)
ndcg_5 = cornac.metrics.NDCG(k=5)
ndcg_10 = cornac.metrics.NDCG(k=10)
auc = cornac.metrics.AUC()

In [6]:
# We created a simple model before so let us try and run a grid search for hyperparameter tuning

# Define the parameters we want to tune and their values
n_epochs_domain = Discrete(name="k", values=[5,10,15])
lr_domain = Discrete("learning_rate", values=[0.001, 0.005, 0.01, 0.05])
reg_mf_domain = Discrete("lambda_reg", values=[0.01, 0.05, 0.1, 0.5])
search_domain = [n_epochs_domain, lr_domain, reg_mf_domain]

# Define the evaluation methods that will be used for the grid search. For this part we'll use a normal 80-10-10 
# train-val-test split.

print("ML100K...")
ml_train_test = RatioSplit(
    data=movielens_data,
    val_size=0.1,
    test_size=0.1,
    rating_threshold=1.0, # This parameter is the threshold used for ranking metrics
    verbose=True
)

# We will run the Grid Search for the measures NDCG and Recall
# This is done for time/performance constraints and also because BPR concentrates on ranking. 
# We will only use the Movielens dataset for the grid search (since the two datasets are similar)

# Define the GridSearch objects
ndcg_gs = GridSearch(model=bpr_model, space=search_domain, metric=ndcg_10, eval_method=ml_train_test)
rec_gs = GridSearch(model=bpr_model, space=search_domain, metric=rec_10, eval_method=ml_train_test)

## Define GridSearch for Movielens
bpr_gridsearch = cornac.Experiment(
    eval_method=ml_train_test,
    models=[ndcg_gs, rec_gs],    
    metrics=[ndcg_10, rec_10],
)

ML100K...
rating_threshold = 1.0
exclude_unknowns = True
---
Training data:
Number of users = 943
Number of items = 1658
Number of ratings = 80000
Max rating = 1.0
Min rating = 1.0
Global mean = 1.0
---
Test data:
Number of users = 922
Number of items = 1227
Number of ratings = 9986
Number of unknown users = 0
Number of unknown items = 0
---
Validation data:
Number of users = 925
Number of items = 1242
Number of ratings = 9986
---
Total users = 943
Total items = 1658


In [7]:
## RUN Grisearch for Movielens
gridsearch_results = bpr_gridsearch.run()


[GridSearch_BPR] Training started!

[GridSearch_BPR] Evaluation started!




[GridSearch_BPR] Training started!

[GridSearch_BPR] Evaluation started!




VALIDATION:
...
               | NDCG@10 | Recall@10 | Time (s)
-------------- + ------- + --------- + --------
GridSearch_BPR |  0.0963 |    0.1071 |   0.8610
GridSearch_BPR |  0.0902 |    0.1075 |   1.3693

TEST:
...
               | NDCG@10 | Recall@10 | Train (s) | Test (s)
-------------- + ------- + --------- + --------- + --------
GridSearch_BPR |  0.0966 |    0.1074 |   86.6429 |   0.9885
GridSearch_BPR |  0.0984 |    0.1112 |   89.7593 |   1.6706



In [8]:
print("Best params based on the NDCG@10 evaluated Grid Search:", ndcg_gs.best_params)
print()
print("Best params based on the Recall@10 evaluated Grid Search:", rec_gs.best_params)

Best params based on the NDCG@10 evaluated Grid Search: {'k': 5, 'lambda_reg': 0.01, 'learning_rate': 0.01}

Best params based on the Recall@10 evaluated Grid Search: {'k': 15, 'lambda_reg': 0.01, 'learning_rate': 0.01}


# 5-fold Cross Validation

In [9]:
# Use the model parameters that were found to work best by the grid search
bpr_model = cornac.models.BPR(
    k=10,
    learning_rate=0.05,
    lambda_reg=0.05
)

In [10]:
# Here we are creating the cross validation procedures for the evaluation. As we can see from the parameters
# we will be running 5f CV on our model on both datasets. The objects that we construct here will be used
# in the cornac experiments in the next cell.
print("ML100K...")
ml_cv = CrossValidation(
    data=movielens_data,
    n_folds=5,
    rating_threshold=1.0, # This parameter is the threshold used for ranking metrics
    seed = 0,
    verbose=True
)
print()
print("PDA...")
pda_cv = CrossValidation(
    data=pda_data,
    n_folds=5,
    rating_threshold=1.0,
    seed = 0,
    verbose=True
)

ML100K...
rating_threshold = 1.0
exclude_unknowns = True

PDA...
rating_threshold = 1.0
exclude_unknowns = True


In [11]:
# RUN 5-fold cross validation w/ Grid Search on Movielens
ml_bpr = cornac.Experiment(
    eval_method=ml_cv,
    models=[bpr_model],    
    metrics=[ndcg_5, ndcg_10, pre_5, pre_10, rec_5, rec_10, auc],
)
ml_bpr.run()

Fold: 1
---
Training data:
Number of users = 943
Number of items = 1648
Number of ratings = 80000
Max rating = 1.0
Min rating = 1.0
Global mean = 1.0
---
Test data:
Number of users = 943
Number of items = 1382
Number of ratings = 19966
Number of unknown users = 0
Number of unknown items = 0
---
Validation data:
Number of users = 943
Number of items = 1382
Number of ratings = 19966
---
Total users = 943
Total items = 1648

[BPR] Training started!

[BPR] Evaluation started!



Fold: 2
---
Training data:
Number of users = 943
Number of items = 1652
Number of ratings = 80000
Max rating = 1.0
Min rating = 1.0
Global mean = 1.0
---
Test data:
Number of users = 942
Number of items = 1371
Number of ratings = 19967
Number of unknown users = 0
Number of unknown items = 0
---
Validation data:
Number of users = 942
Number of items = 1371
Number of ratings = 19967
---
Total users = 943
Total items = 1652

[BPR] Training started!

[BPR] Evaluation started!



Fold: 3
---
Training data:
Number of users = 943
Number of items = 1651
Number of ratings = 80000
Max rating = 1.0
Min rating = 1.0
Global mean = 1.0
---
Test data:
Number of users = 940
Number of items = 1390
Number of ratings = 19965
Number of unknown users = 0
Number of unknown items = 0
---
Validation data:
Number of users = 940
Number of items = 1390
Number of ratings = 19965
---
Total users = 943
Total items = 1651

[BPR] Training started!

[BPR] Evaluation started!



Fold: 4
---
Training data:
Number of users = 943
Number of items = 1656
Number of ratings = 80000
Max rating = 1.0
Min rating = 1.0
Global mean = 1.0
---
Test data:
Number of users = 943
Number of items = 1397
Number of ratings = 19969
Number of unknown users = 0
Number of unknown items = 0
---
Validation data:
Number of users = 943
Number of items = 1397
Number of ratings = 19969
---
Total users = 943
Total items = 1656

[BPR] Training started!

[BPR] Evaluation started!



Fold: 5
---
Training data:
Number of users = 943
Number of items = 1646
Number of ratings = 80000
Max rating = 1.0
Min rating = 1.0
Global mean = 1.0
---
Test data:
Number of users = 942
Number of items = 1383
Number of ratings = 19959
Number of unknown users = 0
Number of unknown items = 0
---
Validation data:
Number of users = 942
Number of items = 1383
Number of ratings = 19959
---
Total users = 943
Total items = 1646

[BPR] Training started!

[BPR] Evaluation started!




TEST:
...
[BPR]
       |    AUC | NDCG@10 | NDCG@5 | Precision@10 | Precision@5 | Recall@10 | Recall@5 | Train (s) | Test (s)
------ + ------ + ------- + ------ + ------------ + ----------- + --------- + -------- + --------- + --------
Fold 0 | 0.9263 |  0.1432 | 0.1357 |       0.1230 |      0.1302 |    0.1024 |   0.0549 |    1.1242 |   2.1431
Fold 1 | 0.9265 |  0.1415 | 0.1305 |       0.1266 |      0.1280 |    0.1033 |   0.0520 |    1.1832 |   1.9369
Fold 2 | 0.9251 |  0.1397 | 0.1308 |       0.1234 |      0.1264 |    0.0979 |   0.0518 |    1.1720 |   1.9822
Fold 3 | 0.9274 |  0.1511 | 0.1446 |       0.1317 |      0.1370 |    0.1018 |   0.0533 |    1.1787 |   1.9836
Fold 4 | 0.9258 |  0.1473 | 0.1397 |       0.1282 |      0.1361 |    0.1045 |   0.0561 |    1.0972 |   1.9695
------ + ------ + ------- + ------ + ------------ + ----------- + --------- + -------- + --------- + --------
Mean   | 0.9262 |  0.1445 | 0.1362 |       0.1266 |      0.1315 |    0.1020 |   0.0536 |    1.1511 |  

In [12]:
# RUN 5-fold cross validation on PDA
pda_bpr = cornac.Experiment(
    eval_method=pda_cv,
    models=[bpr_model],    
    metrics=[ndcg_5, ndcg_10, pre_5, pre_10, rec_5, rec_10, auc],
)
pda_bpr.run()

Fold: 1
---
Training data:
Number of users = 5679
Number of items = 1823
Number of ratings = 376569
Max rating = 1.0
Min rating = 1.0
Global mean = 1.0
---
Test data:
Number of users = 5313
Number of items = 1780
Number of ratings = 94116
Number of unknown users = 0
Number of unknown items = 0
---
Validation data:
Number of users = 5313
Number of items = 1780
Number of ratings = 94116
---
Total users = 5679
Total items = 1823

[BPR] Training started!

[BPR] Evaluation started!



Fold: 2
---
Training data:
Number of users = 5685
Number of items = 1821
Number of ratings = 376569
Max rating = 1.0
Min rating = 1.0
Global mean = 1.0
---
Test data:
Number of users = 5294
Number of items = 1791
Number of ratings = 94127
Number of unknown users = 0
Number of unknown items = 0
---
Validation data:
Number of users = 5294
Number of items = 1791
Number of ratings = 94127
---
Total users = 5685
Total items = 1821

[BPR] Training started!

[BPR] Evaluation started!



Fold: 3
---
Training data:
Number of users = 5684
Number of items = 1823
Number of ratings = 376568
Max rating = 1.0
Min rating = 1.0
Global mean = 1.0
---
Test data:
Number of users = 5328
Number of items = 1798
Number of ratings = 94129
Number of unknown users = 0
Number of unknown items = 0
---
Validation data:
Number of users = 5328
Number of items = 1798
Number of ratings = 94129
---
Total users = 5684
Total items = 1823

[BPR] Training started!

[BPR] Evaluation started!



Fold: 4
---
Training data:
Number of users = 5684
Number of items = 1822
Number of ratings = 376569
Max rating = 1.0
Min rating = 1.0
Global mean = 1.0
---
Test data:
Number of users = 5303
Number of items = 1787
Number of ratings = 94125
Number of unknown users = 0
Number of unknown items = 0
---
Validation data:
Number of users = 5303
Number of items = 1787
Number of ratings = 94125
---
Total users = 5684
Total items = 1822

[BPR] Training started!

[BPR] Evaluation started!



Fold: 5
---
Training data:
Number of users = 5682
Number of items = 1820
Number of ratings = 376569
Max rating = 1.0
Min rating = 1.0
Global mean = 1.0
---
Test data:
Number of users = 5281
Number of items = 1799
Number of ratings = 94121
Number of unknown users = 0
Number of unknown items = 0
---
Validation data:
Number of users = 5281
Number of items = 1799
Number of ratings = 94121
---
Total users = 5682
Total items = 1820

[BPR] Training started!

[BPR] Evaluation started!




TEST:
...
[BPR]
       |    AUC | NDCG@10 | NDCG@5 | Precision@10 | Precision@5 | Recall@10 | Recall@5 | Train (s) | Test (s)
------ + ------ + ------- + ------ + ------------ + ----------- + --------- + -------- + --------- + --------
Fold 0 | 0.8862 |  0.1095 | 0.1026 |       0.0904 |      0.0967 |    0.0855 |   0.0475 |    6.6819 |  10.7381
Fold 1 | 0.8842 |  0.1089 | 0.1038 |       0.0905 |      0.0985 |    0.0832 |   0.0456 |    6.8926 |  11.4937
Fold 2 | 0.8829 |  0.1054 | 0.0982 |       0.0868 |      0.0913 |    0.0813 |   0.0442 |    6.6394 |  10.8333
Fold 3 | 0.8815 |  0.1100 | 0.1050 |       0.0903 |      0.0987 |    0.0829 |   0.0452 |    7.4389 |  10.4338
Fold 4 | 0.8824 |  0.1138 | 0.1069 |       0.0930 |      0.0989 |    0.0870 |   0.0487 |    6.7097 |  10.4109
------ + ------ + ------- + ------ + ------------ + ----------- + --------- + -------- + --------- + --------
Mean   | 0.8834 |  0.1095 | 0.1033 |       0.0902 |      0.0968 |    0.0840 |   0.0462 |    6.8725 |  

# Results

In [13]:
# Export results for both models
print("ML100 Results")
for entry in ml_bpr.result:
    results_dict = entry[0].metric_avg_results
    ml_results_temp_df = pd.DataFrame(results_dict, index=['ML100-BPR' for key in results_dict.keys()])
    ml_results_df = ml_results_temp_df.iloc[0, :].T
    print(ml_results_df)

ML100 Results
AUC             0.926284
NDCG@10         0.143171
NDCG@5          0.135671
Precision@10    0.123012
Precision@5     0.130223
Recall@10       0.102428
Recall@5        0.054893
Train (s)       1.124155
Test (s)        2.143086
Name: ML100-BPR, dtype: float64


In [14]:
print("PDA2018 Results")
for entry in pda_bpr.result:
    results_dict = entry[0].metric_avg_results
    pda_results_temp_df = pd.DataFrame(results_dict, index=['PDA-BPR' for key in results_dict.keys()])
    pda_results_df = pda_results_temp_df.iloc[0, :].T
    print(pda_results_df)

PDA2018 Results
AUC              0.886181
NDCG@10          0.109502
NDCG@5           0.102628
Precision@10     0.090401
Precision@5      0.096744
Recall@10        0.085466
Recall@5         0.047545
Train (s)        6.681921
Test (s)        10.738146
Name: PDA-BPR, dtype: float64


In [15]:
# Export data
final_results_df = ml_results_df.append(pda_results_df)
final_results_df.to_csv("../data/bpr_results.csv")